## Validation

In [1]:
from datetime import date, datetime, timedelta
import pyspark.sql.functions as F
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import col, collect_set, row_number, size
from pyspark.sql.window import Window


def get_data_source(spark: SparkSession, table_name: str) -> DataFrame:
    return spark.sql(f"select * from {table_name}")


def daily_partitions(column_name, start_date, end_date):
    return col(column_name).between(start_date, end_date)


def get_tecton_sessions(spark, df):
    df1 = (
        df
        .groupBy("visit_guid")
        .agg(collect_set("shopper_id").alias("shopper_ids"))
    )
    df2 = (
        df
        .withColumn("rownum", row_number().over(Window.partitionBy("visit_guid").orderBy("session_begin_ts")))
        .filter(col("rownum") == 1)
        .drop("rownum", "shopper_id")
    )
    return df1.join(df2, "visit_guid")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1669225020316_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
tecton = get_data_source(spark, "tecton.shopper_traffic_sessions")
tecton = get_tecton_sessions(spark, tecton)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
uds = get_data_source(spark, "dp_enterprise.uds_traffic_session")
uds = uds.filter(col("shopper_ids").isNotNull() & (size("shopper_ids") > 0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Row count deltas

In [4]:
def one_week(df, start_date):
    return df.filter(daily_partitions("session_date", start_date, start_date + timedelta(days=6)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def show_counts(start_date):
    tecton_counts = one_week(tecton, start_date).groupBy("session_date").count()
    uds_counts = one_week(uds, start_date).groupBy("session_date").count()
    counts = (
        tecton_counts.alias("tecton")
        .join(uds_counts.alias("uds"), "session_date")
        .withColumn("delta", col("tecton.count") / col("uds.count"))
        .orderBy("session_date")
    )
    counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
show_counts(date(2020, 11, 1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------+-------+------------------+
|session_date|  count|  count|             delta|
+------------+-------+-------+------------------+
|  2020-11-01| 691342| 664555|1.0403081761479487|
|  2020-11-02|1078039|1019184|1.0577471781346646|
|  2020-11-03|1040975| 982036|1.0600171480475258|
|  2020-11-04|1044663| 985367| 1.060176563655978|
|  2020-11-05|1038071| 977926|1.0615026085818355|
|  2020-11-06| 942888| 889433|1.0601000862347136|
|  2020-11-07| 667047| 639718|1.0427203861701562|
+------------+-------+-------+------------------+

In [7]:
show_counts(date(2020, 11, 8))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------+-------+------------------+
|session_date|  count|  count|             delta|
+------------+-------+-------+------------------+
|  2020-11-08| 695103| 666614|1.0427368762132208|
|  2020-11-09|1104266|1038029|1.0638103559727137|
|  2020-11-10|1101246|1034689|1.0643256089510955|
|  2020-11-11|1075423|1010333|1.0644243036701762|
|  2020-11-12|1076873|1010746|1.0654239541882926|
|  2020-11-13| 908976| 850907| 1.068243650598714|
|  2020-11-14| 521034| 491676|1.0597100529617065|
+------------+-------+-------+------------------+

In [8]:
show_counts(date(2020, 12, 1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------+------+------------------+
|session_date|  count| count|             delta|
+------------+-------+------+------------------+
|  2020-12-01|1014370|935798| 1.083962564570559|
|  2020-12-02|1006644|929259|1.0832760296106898|
|  2020-12-03| 965607|890213|1.0846920905446225|
|  2020-12-04| 865936|798226|1.0848256007697068|
|  2020-12-05| 591516|556456|1.0630058800695832|
|  2020-12-06| 603048|567926|1.0618425639960136|
|  2020-12-07|1008781|928469|1.0864993877016895|
+------------+-------+------+------------------+

In [9]:
show_counts(date(2021, 1, 1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------+------+------------------+
|session_date|  count| count|             delta|
+------------+-------+------+------------------+
|  2021-01-01| 543668|515067|1.0555286982081942|
|  2021-01-02| 610666|578858|1.0549495731250151|
|  2021-01-03| 636757|602471| 1.056908963253003|
|  2021-01-04|1068487|988059|1.0813999973685782|
|  2021-01-05|1038167|959635|  1.08183528112251|
|  2021-01-06| 995410|920759|1.0810755040135367|
|  2021-01-07| 931538|862121|1.0805188598816176|
+------------+-------+------+------------------+

In [10]:
show_counts(date(2021, 2, 1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------+-------+------------------+
|session_date|  count|  count|             delta|
+------------+-------+-------+------------------+
|  2021-02-01|1100561|1009862| 1.089813261613963|
|  2021-02-02|1058549| 968433|1.0930534172214288|
|  2021-02-03|1039776| 950825|1.0935513895827307|
|  2021-02-04|1032687| 946583|1.0909629689102804|
|  2021-02-05| 903826| 827411|1.0923543438508794|
|  2021-02-06| 606359| 566925| 1.069557701636019|
|  2021-02-07| 605809| 567409|1.0676760502565168|
+------------+-------+-------+------------------+

In [11]:
show_counts(date(2021, 3, 1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------+-------+------------------+
|session_date|  count|  count|             delta|
+------------+-------+-------+------------------+
|  2021-03-01|1093891| 995080|1.0992995538047192|
|  2021-03-02|1116799|1020571| 1.094288393458172|
|  2021-03-03|1077431| 982801|1.0962860233149947|
|  2021-03-04|1021435| 927341|1.1014664508524912|
|  2021-03-05| 923600| 840951|1.0982803992146986|
|  2021-03-06| 590887| 548881|1.0765302497262612|
|  2021-03-07| 605347| 563062|1.0750983017855937|
+------------+-------+-------+------------------+

### Column-by-column deltas

In [12]:
import pandas as pd
import pyspark.sql.functions as F
from functools import reduce

def get_deltas(tecton, uds, session_date):
    tecton = tecton.filter(col("session_date") == session_date).alias("tecton")
    uds = uds.filter(col("session_date") == session_date).alias("uds")
    both = tecton.join(uds, "visit_guid").cache()

    columns = [
        "session_begin_ts",
        "page_views_qty",
        "ip_country_code",
        "repeat_visitor_flag",
        "referring_domain_name",
        "site_country_code",
        "site_language_code",
        "orders_qty",
        "new_orders_qty",
        "renewal_orders_qty",
        "new_product_purchased_flag",
        "renewal_product_purchased_flag",
        "session_free_trial_signup_flag",
        "session_viewed_help_page_flag",
        "session_bounce_flag",
        "session_purchase_flag",
    ]
    delta_list = [
        (name, both.filter(col(f"tecton.{name}") != col(f"uds.{name}")).count()) for name in columns
    ]
    delta_list.append(("extra_shopper_ids", both.filter(col("tecton.extra_shopper_ids") != (size("uds.shopper_ids") - 1)).count()))
    delta_list.append(("page_names", both.filter(size("tecton.page_names") != size("uds.fully_qualified_page_names")).count()))
    # receipt_price_usd_amt +/- $0.01 to allow for rounding
    delta_list.append(("receipt_price_usd_amt", both.filter(F.abs(col("tecton.receipt_price_usd_amt") - col("uds.receipt_price_amt")) > 0.01).count()))
    # session_duration_seconds +/- 1 sec to allow for rounding
    delta_list.append(("session_duration_seconds", both.filter(F.abs(col("tecton.session_duration_seconds") - col("uds.session_duration_seconds")) > 1).count()))
    deltas = pd.DataFrame(delta_list, columns=["column", "delta"])
    deltas["pct"] = deltas["delta"] / tecton.count()
    return deltas

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
get_deltas(tecton, uds, "2020-11-01")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                            column   delta       pct
0                 session_begin_ts    6482  0.009376
1                   page_views_qty   20369  0.029463
2                  ip_country_code       6  0.000009
3              repeat_visitor_flag       0  0.000000
4            referring_domain_name    6465  0.009351
5                site_country_code       8  0.000012
6               site_language_code       2  0.000003
7                       orders_qty    1072  0.001551
8                   new_orders_qty   13785  0.019939
9               renewal_orders_qty     488  0.000706
10      new_product_purchased_flag     724  0.001047
11  renewal_product_purchased_flag     163  0.000236
12  session_free_trial_signup_flag     162  0.000234
13   session_viewed_help_page_flag       0  0.000000
14             session_bounce_flag   35913  0.051947
15           session_purchase_flag     362  0.000524
16               extra_shopper_ids       7  0.000010
17                      page_names  396074  0.

In [14]:
get_deltas(tecton, uds, "2020-11-02")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                            column   delta           pct
0                 session_begin_ts   13887  1.288172e-02
1                   page_views_qty   40794  3.784093e-02
2                  ip_country_code      10  9.276102e-06
3              repeat_visitor_flag       1  9.276102e-07
4            referring_domain_name   12341  1.144764e-02
5                site_country_code      11  1.020371e-05
6               site_language_code       6  5.565661e-06
7                       orders_qty    1721  1.596417e-03
8                   new_orders_qty   18585  1.723964e-02
9               renewal_orders_qty     795  7.374501e-04
10      new_product_purchased_flag    1232  1.142816e-03
11  renewal_product_purchased_flag     253  2.346854e-04
12  session_free_trial_signup_flag     226  2.096399e-04
13   session_viewed_help_page_flag       0  0.000000e+00
14             session_bounce_flag   45515  4.222018e-02
15           session_purchase_flag     581  5.389415e-04
16               extra_shopper_

In [15]:
get_deltas(tecton, uds, "2020-11-03")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                            column   delta           pct
0                 session_begin_ts   13511  1.297918e-02
1                   page_views_qty   38420  3.690771e-02
2                  ip_country_code       6  5.763827e-06
3              repeat_visitor_flag       1  9.606379e-07
4            referring_domain_name   11769  1.130575e-02
5                site_country_code      17  1.633084e-05
6               site_language_code       7  6.724465e-06
7                       orders_qty    1945  1.868441e-03
8                   new_orders_qty   17991  1.728284e-02
9               renewal_orders_qty     838  8.050145e-04
10      new_product_purchased_flag    1349  1.295900e-03
11  renewal_product_purchased_flag     354  3.400658e-04
12  session_free_trial_signup_flag     238  2.286318e-04
13   session_viewed_help_page_flag       0  0.000000e+00
14             session_bounce_flag   44575  4.282043e-02
15           session_purchase_flag     569  5.466029e-04
16               extra_shopper_

### page_names size

In [16]:
def one_day(df, start_date):
    return df.filter(daily_partitions("session_date", start_date, start_date))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
deltas = (
    one_day(tecton, date(2020, 11, 1)).alias("tecton")
    .join(one_day(uds, date(2020, 11, 1)).alias("uds"), "visit_guid")
    .filter(size("tecton.page_names") != size("uds.fully_qualified_page_names"))
)
deltas.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

396074

In [18]:
deltas.select(
    "tecton.visit_guid", size("tecton.page_names"), size("uds.fully_qualified_page_names")
).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+-----------------------+------------------------------------+
|visit_guid                          |size(tecton.page_names)|size(uds.fully_qualified_page_names)|
+------------------------------------+-----------------------+------------------------------------+
|00399a80-6d7e-4380-9e67-12fe77b8faf5|55                     |20                                  |
|005dd466-3afa-4cfa-8a96-ba4f05377d2d|23                     |9                                   |
|0150c028-0946-584f-81b9-1094873336c5|2                      |1                                   |
|017c2473-30d9-58b4-818a-44e9597a8da3|7                      |4                                   |
|02af64ad-e8d9-59d1-a44e-282a641774d5|6                      |3                                   |
|033c2dd1-6fa6-4996-8c43-5d3371a860dd|25                     |7                                   |
|048a64ee-88ce-52db-b5e4-9610b215cad7|21                     |6                                   |


In [19]:
deltas.filter(
    size("tecton.page_names") > size("uds.fully_qualified_page_names")
).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

396072

In [20]:
deltas.withColumn(
    "delta", F.abs(size("tecton.page_names") - size("uds.fully_qualified_page_names"))
).agg(
    F.min("delta"),
    F.max("delta"),
    F.avg("delta"),
    F.stddev("delta"),
    F.percentile_approx("delta", [.99, .95])
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+------------------+------------------+--------------------------------------------------+
|min(delta)|max(delta)|        avg(delta)|stddev_samp(delta)|percentile_approx(delta, array(0.99, 0.95), 10000)|
+----------+----------+------------------+------------------+--------------------------------------------------+
|         1|      9066|10.639675919146422| 32.12894426941615|                                         [110, 40]|
+----------+----------+------------------+------------------+--------------------------------------------------+

In [21]:
one_day(tecton, date(2020, 11, 1)).select(
    F.size("page_names").alias("num_pages")
).agg(
    F.sum("num_pages")
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|sum(num_pages)|
+--------------+
|       7521785|
+--------------+

In [22]:
one_day(uds, date(2020, 11, 1)).select(
    F.size("fully_qualified_page_names").alias("num_pages")
).agg(
    F.sum("num_pages")
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|sum(num_pages)|
+--------------+
|       3271171|
+--------------+

In [23]:
one_day(uds, date(2020, 11, 1)).select(
    F.size("fully_qualified_page_names").alias("num_pages")
).agg(
    F.max("num_pages")
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|max(num_pages)|
+--------------+
|           754|
+--------------+

In [24]:
one_day(tecton, date(2020, 11, 1)).select(
    F.size("page_names").alias("num_pages")
).agg(
    F.max("num_pages")
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|max(num_pages)|
+--------------+
|          9080|
+--------------+